In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, GlobalMaxPooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Example training data
words = [
    "mist", "recycle", "initiative", "texture", "retired", "retirement", "woman", "medicine", "ambition", "shark",
    "consulation", "flawed", "equal", "expenditure", "practice", "dinner", "tendency", "thrust", "taste", "blow",
    "disappointment", "arrangement", "seminar", "penetrate", "battle", "harmony", "import", "captain", "prove", "chalk",
    "window", "knowledge", "orange", "beautiful", "computer", "university", "language", "river", "mountain", "forest",
    "music", "painting", "village", "country", "freedom", "teacher", "student", "pencil", "garden", "library",
    "engine", "science", "future", "memory", "danger", "courage", "ocean", "planet", "galaxy", "universe",
    "holiday", "season", "family", "friendship", "happiness", "darkness", "lightning", "shadow", "morning", "evening",
    "midnight", "thunder", "rainbow", "sunset", "sunrise", "calendar", "history", "government", "president", "village",
    "castle", "palace", "kingdom", "fortune", "wisdom", "justice", "reality", "illusion", "mystery", "puzzle",
    "poetry", "fiction", "character", "dialogue", "chapter", "sentence", "paragraph", "dictionary", "philosophy", "biology"
]

syllables = [
    1, 3, 5, 2, 3, 4, 2, 3, 3, 1,
    4, 1, 2, 4, 2, 2, 3, 1, 1, 1,
    4, 3, 3, 3, 2, 3, 2, 2, 1, 1,
    2, 2, 2, 3, 3, 5, 2, 2, 2, 2,
    2, 2, 2, 2, 2, 2, 2, 2, 2, 3,
    2, 2, 2, 3, 2, 2, 2, 2, 3, 3,
    3, 2, 3, 2, 3, 2, 2, 2, 2, 2,
    2, 2, 2, 2, 2, 3, 3, 3, 3, 2,
    2, 2, 2, 2, 2, 2, 4, 3, 3, 2,
    3, 2, 3, 3, 2, 2, 3, 4, 4, 4
]

# Tokenize words at character level
tokenizer = Tokenizer(char_level=True, lower=True)
tokenizer.fit_on_texts(words)
sequences = tokenizer.texts_to_sequences(words)

# Pad sequences
maxlen = max(len(w) for w in words)
X = pad_sequences(sequences, maxlen=maxlen, padding='post')

# Define max syllable count (important for classification range)
max_syllables = max(syllables)

# Convert to categorical (one-hot labels)
y = tf.keras.utils.to_categorical(syllables, num_classes=max_syllables + 1)

# Define model
model = Sequential([
    Embedding(input_dim=len(tokenizer.word_index) + 1,
              output_dim=32,
              input_length=maxlen),
    LSTM(64, return_sequences=True),
    GlobalMaxPooling1D(),
    Dense(32, activation='relu'),
    Dense(32, activation='relu'),
    Dense(max_syllables + 1, activation='softmax')  # classification output
])

# Compile model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train model
model.fit(X, y, epochs=200, verbose=1)

2025-09-09 15:08:37.169008: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-09 15:08:42.648504: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-09 15:08:42.652367: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Epoch 1/200
4/4 [==============================] - 3s 44ms/step - loss: 1.7820 - accuracy: 0.4900
Epoch 2/200
4/4 [==============================] - 0s 81ms/step - loss: 1.7551 - accuracy: 0.5400
Epoch 3/200
4/4 [==============================] - 0s 22ms/step - loss: 1.7197 - accuracy: 0.5400
Epoch 4/200
4/4 [==============================] - 0s 24ms/step - loss: 1.6706 - accuracy: 0.5400
Epoch 5/200
4/4 [==============================] - 0s 40ms/step - loss: 1.5892 - accuracy: 0.5400
Epoch 6/200
4/4 [==============================] - 0s 31ms/step - loss: 1.4669 - accuracy: 0.5400
Epoch 7/200
4/4 [==============================] - 0s 62ms/step - loss: 1.2687 - accuracy: 0.5400
Epoch 8/200
4/4 [==============================] - 0s 36ms/step - loss: 1.2119 - accuracy: 0.5400
Epoch 9/200
4/4 [==============================] - 0s 57ms/step - loss: 1.1929 - accuracy: 0.5400
Epoch 10/200
4/4 [==============================] - 0s 16ms/step - loss: 1.1772 - accuracy: 0.5400
Epoch 11/200
4/4 [=

In [2]:
# Test prediction
test_words = ["goat", "hardware", "benefit", "tree", "unit", "sight", "touch", "voter", "network", "party"]
test_seq = tokenizer.texts_to_sequences(test_words)
test_X = pad_sequences(test_seq, maxlen=maxlen, padding='post')
preds = model.predict(test_X)

# Convert softmax to integer syllable predictions
pred_classes = np.argmax(preds, axis=1)

for w, p in zip(test_words, pred_classes):
    print(f"{w} → predicted syllables: {p}")

1/1 [==============================] - 0s 430ms/step
goat → predicted syllables: 2
hardware → predicted syllables: 2
benefit → predicted syllables: 2
tree → predicted syllables: 2
unit → predicted syllables: 2
sight → predicted syllables: 2
touch → predicted syllables: 1
voter → predicted syllables: 1
network → predicted syllables: 2
party → predicted syllables: 2
